# **CSCE 5218 / CSCE 4930 Deep Learning**

# **HW1a The Perceptron** (20 pt)


In [110]:
# Get the datasets
!curl http://huang.eng.unt.edu/CSCE-5218/test.dat
!curl http://huang.eng.unt.edu/CSCE-5218/train.dat


A1	A2	A3	A4	A5	A6	A7	A8	A9	A10	A11	A12	A13
1	1	1	1	0	0	1	1	0	0	0	1	1	0
0	0	0	1	0	0	1	1	0	1	0	0	1	0
0	1	1	1	0	1	1	1	1	0	0	0	1	0
0	1	1	0	1	0	1	1	1	0	1	0	1	0
0	1	0	0	0	1	0	1	0	1	0	0	1	0
0	1	1	0	0	1	1	1	1	1	1	0	1	0
0	1	1	1	0	0	1	1	0	0	0	1	1	0
0	1	0	0	1	0	0	1	1	0	1	1	1	0
1	1	1	1	0	0	1	1	0	0	0	0	1	0
1	0	1	1	1	1	1	0	0	0	0	0	1	0
1	0	0	0	1	0	0	1	1	0	1	1	1	0
0	0	1	0	0	1	1	0	0	0	0	1	1	0
0	1	0	1	1	1	0	0	0	0	0	1	1	0
0	1	0	0	0	1	1	1	0	1	1	1	1	0
1	1	1	0	1	0	0	1	0	1	1	0	1	1
1	0	0	1	0	0	1	1	0	0	1	1	1	0
0	0	0	1	0	1	1	0	1	1	1	1	1	0
1	0	1	0	1	1	0	0	0	1	0	0	1	0
0	1	1	0	1	0	1	0	1	1	1	0	1	1
0	1	0	0	1	1	1	1	1	1	0	0	1	0
1	1	0	1	0	1	1	0	1	0	0	0	1	0
1	0	1	1	0	1	0	0	0	0	1	1	1	1
1	0	1	1	1	1	0	1	1	0	1	0	1	1
1	0	0	1	0	0	0	1	0	1	0	1	1	1
1	1	0	1	1	0	1	1	0	0	1	0	1	0
0	0	0	0	1	1	1	0	0	1	0	1	1	0
1	1	1	0	0	0	1	0	0	0	1	1	1	0
1	0	1	0	0	1	0	0	0	0	1	1	1	0
1	0	1	1	1	1	0	0	0	0	1	1	1	0
0	1	0	0	1	1	1	0	1	1	0	1	1	1
0	1	1	1	0	0	1	0	1	1	1	0	1	1
1	0	0	0	0	0	0	1	1	1	1	0	1	1
1	0	1	1	0	1	1	0	0	1	0	1	1	0
1	1	1	0	1	1	0	1	1	0	1	0	1	1
0	1	0

In [111]:
# Take a peek at the datasets
!head train.dat
!head test.dat

head: train.dat: No such file or directory
head: test.dat: No such file or directory


### Build the Perceptron Model

You will need to complete some of the function definitions below.  DO NOT import any other libraries to complete this. 

In [112]:
import math
import itertools
import re


# Corpus reader, all columns but the last one are coordinates;
#   the last column is the label
def read_data(file_name):
    f = open(file_name, 'r')
    data = []
    # Discard header line
    f.readline()
    for instance in f.readlines():
        if not re.search('\t', instance): continue
        instance = list(map(int, instance.strip().split('\t')))
        # Add a dummy input so that w0 becomes the bias
        instance = [-1] + instance
        data += [instance]
    return data


def dot_product(array1, array2):
    # dot product of array 1 and array 2
    dot_product = 0
    for i,j in zip(array1,array2):
        dot_product = dot_product + (i * j)
    return dot_product


def sigmoid(x):
    # outpout of sigmoid function on x
    sig = (1.0 / (1.0 + math.exp(-1.0*x)))
    return sig

# The output of the model, which for the perceptron is 
# the sigmoid function applied to the dot product of 
# the instance and the weights
def output(weight, instance):
    # output of the model
    output = sigmoid(dot_product(weight, instance))
    return output

# Predict the label of an instance; this is the definition of the perceptron
# you should output 1 if the output is >= 0.5 else output 0
def predict(weights, instance):
    # prediction of the model
    if output(weights, instance) >= 0.5:
        return 1
    else:
        return 0


# Accuracy = percent of correct predictions
def get_accuracy(weights, instances):
    # You do not to write code like this, but get used to it
    correct = sum([1 if predict(weights, instance) == instance[-1] else 0
                   for instance in instances])
    return correct * 100 / len(instances)


# Train a perceptron with instances and hyperparameters:
#       lr (learning rate) 
#       epochs
# The implementation comes from the definition of the perceptron
#
# Training consists on fitting the parameters which are the weights
# that's the only thing training is responsible to fit
# (recall that w0 is the bias, and w1..wn are the weights for each coordinate)
#
# Hyperparameters (lr and epochs) are given to the training algorithm
# We are updating weights in the opposite direction of the gradient of the error,
# so with a "decent" lr we are guaranteed to reduce the error after each iteration.
def train_perceptron(instances, lr, epochs):

    # weight with length of training data
    weights = [0] * (len(instances[0])-1)

    for _ in range(epochs):
        for instance in instances:
            # loop each object in training data for an each epoch
            in_value = dot_product(weights, instance)
            output = sigmoid(in_value)
            error = instance[-1] - output
            # loop each weight and update it for row in epoch
            for i in range(0, len(weights)):
                weights[i] += lr * error * output * (1-output) * instance[i]

    return weights

## Run it

In [113]:
instances_tr = read_data("train.dat.txt")
instances_te = read_data("test.dat.txt")
lr = 0.005
epochs = 5
weights = train_perceptron(instances_tr, lr, epochs)
accuracy = get_accuracy(weights, instances_te)
print(f"#tr: {len(instances_tr):3}, epochs: {epochs:3}, learning rate: {lr:.3f}; "
      f"Accuracy (test, {len(instances_te)} instances): {accuracy:.1f}")

#tr: 400, epochs:   5, learning rate: 0.005; Accuracy (test, 100 instances): 68.0


## Questions

Answer the following questions. Include your implementation and the output for each question.



### Question 1

In `train_perceptron(instances, lr, epochs)`, we have the follosing code:
```
in_value = dot_product(weights, instance)
output = sigmoid(in_value)
error = instance[-1] - output
```

Why don't we have the following code snippet instead?
```
output = predict(weights, instance)
error = instance[-1] - output
```

#### TODO Add your answer here (text only)

The output of both code snippet is same

#tr: 400, epochs:   5, learning rate: 0.005; Accuracy (test, 100 instances): 32.0

but when we add print for 
"weights = train_perceptron(instances_tr, lr, epochs)"
The weights are not accurate with second code snippet:
```
output = predict(weights, instance)
error = instance[-1] - output
```
output for weights:  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

for first code snippet - 
```
in_value = dot_product(weights, instance)
output = sigmoid(in_value)
error = instance[-1] - output
```
output for weights:  [0.08016249451473897, -0.018803740641657435, -0.02722022427977521, -0.02235226664042487, -0.02336209277151821, -0.030082783050668655, -0.022046559929893565, -0.025849811991035504, -0.018463944506982403, -0.02095125584232907, -0.019019377071074716, -0.013626128855073924, -0.02402019869311934, -0.08016249451473897]

to get exact values for the weight we are using first code snippet. where in predict we are directly returning 0 or 1 .

### Question 2
Train the perceptron with the following hyperparameters and calculate the accuracy with the test dataset.

```
tr_percent = [5, 10, 25, 50, 75, 100] # percent of the training dataset to train with
num_epochs = [5, 10, 20, 50, 100]              # number of epochs
lr = [0.005, 0.01, 0.05]              # learning rate
```

TODO: Write your code below and include the output at the end of each training loop (NOT AFTER EACH EPOCH)
of your code.The output should look like the following:
```
# tr:  20, epochs:   5, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
# tr:  20, epochs:  10, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
# tr:  20, epochs:  20, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
[and so on for all the combinations]
```
You will get different results with different hyperparameters.

#### TODO Add your answer here (code and output in the format above) 



In [114]:
instances_tr = read_data("train.dat.txt")
instances_te = read_data("test.dat.txt")
tr_percent = [5, 10, 25, 50, 75, 100] # percent of the training dataset to train with
num_epochs = [5, 10, 20, 50, 100]     # number of epochs
lr_array = [0.005, 0.01, 0.05]        # learning rate

for lr in lr_array:
    for tr_size in tr_percent:
        for epochs in num_epochs:
            size =  round(len(instances_tr)*tr_size/100)
            pre_instances = instances_tr[0:size]
            weights = train_perceptron(pre_instances, lr, epochs)
            accuracy = get_accuracy(weights, instances_te)
            print(f"#tr: {len(pre_instances):0}, epochs: {epochs:3}, learning rate: {lr:.3f}; "
            f"Accuracy (test, {len(instances_te)} instances): {accuracy:.1f}")

#tr: 20, epochs:   5, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
#tr: 20, epochs:  10, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
#tr: 20, epochs:  20, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
#tr: 20, epochs:  50, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
#tr: 20, epochs: 100, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
#tr: 40, epochs:   5, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
#tr: 40, epochs:  10, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
#tr: 40, epochs:  20, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
#tr: 40, epochs:  50, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
#tr: 40, epochs: 100, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
#tr: 100, epochs:   5, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
#tr: 100, epochs:  10, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
#tr: 100, epochs:  20, lea

### Question 3
Write a couple paragraphs interpreting the results with all the combinations of hyperparameters. Drawing a plot will probably help you make a point. In particular, answer the following:
- A. Do you need to train with all the training dataset to get the highest accuracy with the test dataset?
- B. How do you justify that training the second run obtains worse accuracy than the first one (despite the second one uses more training data)?
   ```
#tr: 100, epochs:  20, learning rate: 0.050; Accuracy (test, 100 instances): 71.0
#tr: 200, epochs:  20, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
```
- C. Can you get higher accuracy with additional hyperparameters (higher than `80.0`)?
- D. Is it always worth training for more epochs (while keeping all other hyperparameters fixed)?

#### TODO: Add your answer here (code and text)



### Answer 3:
Hyperparameter optimization is the process of finding the right combination of hyperparameter values to achieve max performance on your data in a Reasonable time.

we can get results by using Grid Search, Random Search optimization:

Grid Search:
 - a widely used method that performs to get the optimal values for a given model using hyperparameter tuning.
 - works by trying all possible combination of parameters you want to try in the model.
 - will take more time to perform the entire search due to this can get very computationally expensive.
 
Random Search:
 - Works bit differently, compared to grid search. 
 - random combinations of the values of the hyperparameters are used to find the best solution for the built models
 - Drawback: sometimes miss important values in the search space.

### 3.A. Do you need to train with all the training dataset to get the highest accuracy with the test dataset?

We don't need to train with all the training datasets to get the highest accuracy with testdata.

- Having more data certainly will increase the accuracy of your model, but at certain point where even adding infinite of data cannot improve any more accuracy. this is called as natural noise of the data.

- Placing too much emphasis on each data point, there needs to be deal with lot of noise, that cause lose sight of whats really important.

- Conclusion: So adding more data points to the training set will not improve the model accuracy. 

### 3.B. How do you justify that training the second run obtains worse accuracy than the first one (despite the second one uses more training data)?

There might be chance of overfitting by providing more training data.

- If the model is trained with too much data, then it could essentially memorize the data, which causes model overfitting, which causes a high error rate for unseen data. Due to this we get wrong predictions and lose focus on important data.

- Sometimes we dont need more training data to get high accuracy


### 3.C. Can you get higher accuracy with additional hyperparameters (higher than 80.0)?

We can not get accuracy higher than 80.0 by adding additional hyperparameters. 
- when model is overfitting(performing well on a training set and poor on test dataset)
- when model is underftting(performing poorly on tyhe training dataset and well on test dataset
- optimizing hypperparameters can help, little tweak can make difference in getting accuracy from 60% to 80%.


### 3.D. Is it always worth training for more epochs (while keeping all other hyperparameters fixed)?

Training for more epochs is worth when setting the number of epochs as high as possible and terminate the training when validation error start increasing.

- Too many epochs can lead to overfitting of the training dataset
- Too few epochs may result in an underfit model
- Early stopping is a method that allows you to specify number of training epochs and stop training once the model performance stops improving on a hold out validation dataset.